### Auspex + QGL, qubit characterization example

In [ ]:
from auspex.exp_factory import QubitExpFactory, quince
import auspex.pulse_calibration as cal
from auspex.single_shot_fidelity import SingleShotFidelityExperiment
from QGL import *
import os
from auspex.instruments.utils import pulse_marker
seq_path = 'C:\\Users\\qlab_user\\Documents\\awg'
ChannelLibrary()
output_notebook()

from auspex.analysis.io import load_from_HDF5
from auspex.analysis.fits import *

In [ ]:
# Define qubits
q1 = QubitFactory('q1')
#q2 = QubitFactory("q2")
#q3 = QubitFactory("q3")
#q4 = QubitFactory("q4")
#q5 = QubitFactory("q5")
q = q1

Measurement settings are found in the following file and the included instruments and filters files

In [ ]:
config.meas_file

Relevant config. directories are also included:
* AWGDir: path of APS sequences
* KernelDir: path of saved kernels for Integrated X6 streams (see filters)
* LogDir: path of experiment and calibration log files
* PulsePrimitiveLibrary: standard/all90 for definition of a pi pulse as X(q) / [X90(q), X90(q)]

Filters can also be displayed and set using Quince:

In [ ]:
quince()

### Pulsed spectroscopy
A single sequence with a control pulse followed by a measurement pulse. When performing cavity spectroscopy, the control pulse can be disabled with the ``specOn`` option set to false. Typically, once the low power cavity is found, the qubit is probed with a long (>3 us) saturating pulse (``specOn = True``). During this stage APS2 on-board modulation is typically set to zero.

In [ ]:
fstart = 6.2e9
fstop = 6.5e9
nsteps = 51

# compiling a sequence in QGL produces the metafile with the info on sequences, APS, axes, and number of segments.
#meta_file = PulsedSpec(q,specOn=False, showPlot=True)

# you can then create an Auspex experiment based on such metafile
#exp = QubitExpFactory.create(meta_file, expname = 'Spec', cw_mode = True)

#### Cavity Spec

In [ ]:
exp = QubitExpFactory.create(PulsedSpec(q1,specOn=False, showPlot=True), expname = 'Spec', cw_mode = True)

# add sweep to determine cavity location
exp.add_qubit_sweep("q1 measure frequency", np.linspace(fstart, fstop, nsteps))

# run sweep
exp.run_sweeps()

# once the cavity location has been precisely determined, one can sweep the APS amplitude 
# this will create a 2D sweep, where the amplitude being swept is the channel scale amplitude in instruments.yml

# exp = QubitExpFactory.create(PulsedSpec(q1,specOn=False, showPlot=True), expname = 'Spec')
# exp.add_qubit_sweep("q1 measure frequency", np.linspace(fstart, fstop, nsteps))
# exp.add_qubit_sweep("q1 measure amplitude", np.linspace(0.1,0.8,8)) 
# run sweep
#exp.run_sweeps()

After the power appropriate power and frequency have been determined, add a non-zero 'autodyne_freq' to the APS2 measure parameters. This will move the LO leakage off resonance from the cavity. A new sweep should be performed to ensure that the cavity is in the correct place (should move by added autodyne frequency).

### Qubit Spec

After finding the low-power cavity frequency, spectroscopy for the qubit is performed.Typically one starts with a wide sweep with high power and long interrogation time. After features appears, one can perform narrower sweeps at lower power. The experiment is the same in all cases, however the user should adjust measure length and amplitude within the qubit measure parameters (measure.yml). The device being swept is the rf generator associated with the control or measure channel.

In [ ]:
# Other examples: 
exp = QubitExpFactory.create(PulsedSpec(q1,specOn=True, showPlot=True), expname = 'Spec')
exp.add_qubit_sweep("q1 control frequency", np.linspace(5e9, 5.2e9, 201)) #sweep qubit drive frequency, 

# run sweep
exp.run_sweeps()

### Rabi
Once you find the resonance in spectroscopy, a good starting point is to measure Rabi oscillations vs frequency to have a sense of driving amplitude and a slightly more accurate qubit frequency. 

In [ ]:
fstart = 5e9
fstop = 5.1e9
nsteps = 51
exp = QubitExpFactory.create(RabiWidth(q,1e-9*np.arange(20, 2000, 40), showPlot=False), expname = 'Rabi_width.h5')
# Alternatively, sweep amplitude:
#exp = QubitExpFactory.create(RabiAmp(q,np.linspace(-1,1,101), showPlot=False), expname = 'Rabi_amp.h5')
#exp.add_qubit_sweep("q1 measure frequency", np.linspace(fstart, fstop, nsteps))
exp.run_sweeps()

# Note that if no sweep is added, one can create and run the experiment in one line:
# exp = QubitExpFactory.run(RabiWidth(q,1e-9*np.arange(20, 2000, 40)), expname = 'Rabi_width.h5')

### Mixer calibration

Calibrate mixer routine. Switches may be present to direct the signal to the SA. In that case, switches are defined within the marker section in `meas.yaml`:

```
markers:
  digitizerTrig: APSII4 12m1
  digitizer2Trig: APSII4 12m4
  switch_M1: APSII1 12m3
  switch_q1: APSII3 12m1
```

In [ ]:
switch_target = 'q1'
switch_type = 'c' # m or c for control or measurement
toggle_switch = True # toggle switch to SA before and after mixer calibration

if switch_type == "m":
    switch_type = "measure"
    switch_name = switch_target.replace("q","M")
elif switch_type == "c":
    switch_type = "control"
    switch_name = switch_target

if toggle_switch:
    pulse_marker('switch_'+switch_name)

exp = QubitExpFactory.calibrate_mixer(switch_target,mixer=switch_type,write_to_file=True, nsteps=21, offset_range = (-0.2,0.2))

if toggle_switch:
    pulse_marker('switch_'+switch_name)

In [ ]:
# You can get the docstrings of any of the above functions using:
? QubitExpFactory.calibrate_mixer

### Readout kernel calibration (optional)

If using an integrated stream (as opposed to demodulated) on the X6, this step is recommended early in the process. Go back to this step for a better calibration once the pulses are tuned 

In [ ]:
# This experiment requires a SingleShotMeasurement filter for q
exp = SingleShotFidelityExperiment(q.label, num_shots=40000, optimize=True)

# You can add sweeps to optimize over any instrument parameter (if the optimize flag above is True)
# example: 
# exp.add_qubit_sweep("q1 measure amplitude", np.linspace(0.2,0.4,8))

exp.run_sweeps()

Once the Filter Kernel has been calculated, switch over it in filters.yml. Change 'simple_kernel' in q1-integrator to 'false.' Then add kernel : [path to filter kernel] to the same instruments. There is a commented example

### Frequency calibration (Ramsey)

Once pulses and qubit frequencies are *roughly* calibrated, one should fine tune them. First, get the qubit frequency correctly. Use shorter delays for larger detunings, longer delays for fine tuning.

In [ ]:
RamseyStart = 0; RamseyStop= 10e-6;RamseyPoints = 101; #sweep for total Ramsey delay
exp = QubitExpFactory.run(Ramsey(q,np.linspace(RamseyStart,RamseyStop,RamseyPoints), TPPIFreq = 0.0e6), expname = 'Ramsey_{}.h5'.format(q.label))

If the qubit frequency is close enough (~<1 MHz detuning), use the calibration routine.

`set_source = True` to calibrate the generator frequency

`set_source = False` to calibrate the qubit pulse modulation instead. 

In [ ]:
cal.calibrate([cal.RamseyCalibration(q.label, set_source = False, delays = np.linspace(0,50,51)*1e-6, added_detuning=0e3, two_freqs = False, AIC=False, quad="real")])

### Pulse calibration

In [ ]:
cal.calibrate([cal.Pi2Calibration(q.label)])
cal.calibrate([cal.PiCalibration(q.label)]) # not necessary if using PulsePrimitiveLibrary: all90

### Coherence measurements

In [ ]:
T1Start = 0e-6; T1Step = 2000e-9; T1Stop = 200e-6;

RamseyStart = 0; RamseyStop= 50e-6;RamseyPoints = 101; #sweep for total Ramsey delay

EchoStart = 10e-6; EchoStop = 100e-6; EchoPoints = 101; #sweep for total Echo delay

#T1
#exp = QubitExpFactory.run(InversionRecovery(q,np.arange(T1Start,T1Stop,T1Step)), expname = 'T1-{}.h5'.format(q.label))

#T2*
#exp = QubitExpFactory.run(Ramsey(q,np.linspace(RamseyStart,RamseyStop,RamseyPoints), TPPIFreq = 0.0e6), expname = 'Ramsey_{}.h5'.format(q.label))

#T2 (periods indicate the number of artificial full oscillations baked into the 2nd pi/2 phase)
#exp = QubitExpFactory.run(HahnEcho(q, np.linspace(EchoStart/2, EchoStop/2, EchoPoints), showPlot = False, periods = 3, calRepeats = 2) , expname = "Echo.h5")

Loading and analyzing data can be done in Qlab.jl (https://github.com/BBN-Q/Qlab.jl) or using the fit functions available in Auspex\analysis\fits.py

### Randomized Benchmarking

In [ ]:
lengths=[2**n for n in range(1,8)]
seqs = create_RB_seqs(1, lengths)
exp = QubitExpFactory.create(SingleQubitRB(q1, seqs), expname='Q1RB')
exp.run_sweeps()

In [ ]:
#data = load_from_HDF5('/home/qlab/data/EXP/180918/Q1RB-0004.h5')[0] # get correct data path

In [ ]:
cdata, xpts = cal_data(data, quad=np.real);

In [ ]:
fit_single_qubit_rb(cdata, lengths, showPlot=True)